In [24]:
import torch
import os
import pandas as pd
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import sys


import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
import torch.optim as optim
import torch.nn.functional as tfunc
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as func

from sklearn.metrics.ranking import roc_auc_score

from torch.utils.data import Dataset
from PIL import Image
from models.chexnet.DensenetModels import DenseNet121
from models.models import ResNet18
from tensorboardX import SummaryWriter
from sklearn.metrics import classification_report, accuracy_score

%load_ext autoreload
%autoreload 2

torch.cuda.current_device()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


0

In [3]:
"""
Read images and corresponding labels.
"""
class ChestXrayDataSet(Dataset):
    def __init__(self, data_dir, image_list_file, diseases, transform=None):
        """
        Args:
            data_dir: path to image directory.
            image_list_file: path to the file containing images
                with corresponding labels.
            transform: optional transform to be applied on a sample.
        """
        image_names = []
        labels = []
        chex_df = pd.read_csv(image_list_file)
        chex_df = chex_df.fillna(0.0)
        if 'train' in image_list_file:
            chex_df = chex_df[:10000]
        if len(diseases) == 1:
            chex_df = chex_df.loc[chex_df['Pleural Effusion'] != -1] #U-Ignore
            
        labels = chex_df.as_matrix(columns=diseases)
        labels = list(labels)

        image_names = chex_df.as_matrix(columns=['Path']).flatten()
        image_names = [os.path.join(data_dir, im_name) for im_name in image_names]

        self.image_names = image_names
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        """
        Args:
            index: the index of item
        Returns:
            image and its labels
        """
        image_name = self.image_names[index]
        image = Image.open(image_name).convert('RGB')
        label = torch.FloatTensor(self.labels[index])
        if self.transform is not None:
            image = self.transform(image)
        return image, label

    def __len__(self):
        return len(self.image_names)

In [4]:
model = ResNet18(1, False).cuda()
model = torch.nn.DataParallel(model).cuda()
checkpoint = torch.load('m-.pth.tar')

In [3]:
model = DenseNet121(1, False).cuda()
model = torch.nn.DataParallel(model).cuda()
checkpoint = torch.load('m-dense.pth.tar')

In [5]:
model.load_state_dict(checkpoint['state_dict'])
del checkpoint

In [6]:
VAL_IMAGE_LIST = './data/CheXpert-v1.0-small/valid.csv'
DATA_DIR = './data'
trBatchSize = 2
transResize = 256
transCrop = 224

#-------------------- SETTINGS: DATA TRANSFORMS, TEN CROPS |VAL|
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

#-------------------- SETTINGS: DATASET BUILDERS |VAL|
transformList = []
transformList.append(transforms.Resize(transResize))
transformList.append(transforms.TenCrop(transCrop))
transformList.append(transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])))
transformList.append(transforms.Lambda(lambda crops: torch.stack([normalize(crop) for crop in crops])))
transformSequence=transforms.Compose(transformList)

datasetVal =   ChestXrayDataSet(data_dir=DATA_DIR, image_list_file=VAL_IMAGE_LIST, diseases=['Pleural Effusion'], transform=transformSequence)
dataLoaderVal = DataLoader(dataset=datasetVal, batch_size=trBatchSize, shuffle=False, num_workers=4, pin_memory=True)





/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [7]:
def computeAUROC (dataGT, dataPRED, classCount):

    outAUROC = []

    datanpGT = dataGT.cpu().numpy()
    datanpPRED = dataPRED.cpu().numpy()

    for i in range(classCount):
        outAUROC.append(roc_auc_score(datanpGT[:, i], datanpPRED[:, i]))

    return outAUROC

In [30]:
def computeClassMetrics(dataGT, dataPRED, classCount):
    classification_metrics = []
    datanpGT = dataGT.cpu().numpy()
    datanpPRED = dataPRED.cpu().numpy()

    for i in range(classCount):
        pred_to_category = datanpPRED[:, i].copy()
        pred_to_category[pred_to_category < 0.5]  = 0
        pred_to_category[pred_to_category != 0]  = 1
        classification_metrics.append(classification_report(datanpGT[:, i], pred_to_category))
    return classification_metrics

In [22]:
def computeAcc(dataGT, dataPRED, classCount):
    acc = []
    datanpGT = dataGT.cpu().numpy()
    datanpPRED = dataPRED.cpu().numpy()

    for i in range(classCount):
        pred_to_category = datanpPRED[:, i].copy()
        pred_to_category[pred_to_category < 0.5]  = 0
        pred_to_category[pred_to_category != 0]  = 1
        acc.append(accuracy_score(datanpGT[:, i], pred_to_category))
    return acc

In [25]:
outGT = torch.FloatTensor().cuda()
outPRED = torch.FloatTensor().cuda()

model.eval()

lossVal = 0
lossValNorm = 0
classCount = 1

losstensorMean = 0
for i, (input, target) in enumerate(dataLoaderVal):

    target = target.cuda()
    outGT = torch.cat((outGT, target), 0)
    loss = torch.nn.BCELoss(size_average = True)

    bs, n_crops, c, h, w = input.size()

    varInput = torch.autograd.Variable(input.view(-1, c, h, w).cuda(), volatile=True)

    out = model(varInput)
    outMean = out.view(bs, n_crops, -1).mean(1)

    outPRED = torch.cat((outPRED, outMean.data), 0)

    varOutput = outPRED
    varTarget = outGT

    losstensor = loss(varOutput, varTarget)
    losstensorMean += losstensor
    lossVal += losstensor.item()
    lossValNorm += 1

outLoss = lossVal / lossValNorm
losstensorMean = losstensorMean / lossValNorm

classMetrics = computeClassMetrics(outGT, outPRED, classCount)
aurocIndividual = computeAUROC(outGT, outPRED, classCount)
aurocMean = np.array(aurocIndividual).mean()
accMean = np.array(computeAcc(outGT, outPRED, classCount)).mean()

/opt/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1.
 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1.
 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0.09878731 0.03087339 0.03040134 0.4432914  0.01663688 0.57143307
 0.42132092 0.03934598 0.03840555 0.08081787 0.8180542  0.02709676
 0.02591118 0.11609687 0.00828594 0.84540147 0.0299581  0.03556917
 0.4658061  0.02118432 0.01321094 0.08392068 0.06361598 0.05721939
 0.07336508 0.03500

0.9105371346858522

In [29]:
print('AUROC:', aurocMean)

AUROC: 0.9105371346858522


In [21]:
for metric in classMetrics:
    print(metric)

              precision    recall  f1-score   support

         0.0       0.85      0.95      0.90       167
         1.0       0.83      0.60      0.70        67

   micro avg       0.85      0.85      0.85       234
   macro avg       0.84      0.77      0.80       234
weighted avg       0.85      0.85      0.84       234



In [28]:
print('Accuracy:', accMean)

Accuracy: 0.8504273504273504
